Imports

In [ ]:
from pathlib import Path

from src.modeling import LatentThinkingModel
from src.model_creation import ModelCreationConfig
from src.utils import hf_download_and_unjson

Variables

In [ ]:
''' Model '''
## Specify EITHER model_path_or_repo and lora_path_or_repo OR a path or repo which contains a model_creation_config.json
model_path_or_repo: str | None = None
lora_path_or_repo: str | None = None
## OR
mcc_path_or_repo: str | None = None

''' New repo id '''
new_hub_repo_id = "user/new_repo_name"

Merge and Push

In [ ]:
if model_path_or_repo is not None and mcc_path_or_repo is not None:
    raise ValueError("Specify ONE of model_path_or_repo and mcc_path_or_repo")

if model_path_or_repo is not None:
    mcc = ModelCreationConfig(path_or_repo=model_path_or_repo, is_lora=bool(lora_path_or_repo), lora_path_or_repo=lora_path_or_repo)
elif mcc_path_or_repo is not None:
    mcc_path = Path(mcc_path_or_repo) / "model_creation_config.json"
    if mcc_path.exists():
        mcc = ModelCreationConfig.load_from_json(mcc_path)
    else:
        mcc = ModelCreationConfig(
            **hf_download_and_unjson(mcc_path_or_repo + "/model_creation_config.json")
        )
else:
    raise ValueError("Not one of model_path_or_repo or mcc_path_or_repo provided")

model, tokenizer = mcc.get_model_and_tokenizer()

model = model.merge_and_unload()

if isinstance(model, LatentThinkingModel):
    model.lora_mode_disable()

model.push_to_hub(new_hub_repo_id, private=True)
tokenizer.push_to_hub(new_hub_repo_id, private=True)